In [140]:
from imports import *

def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}

class WhBase:
    
    pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
    body_chat_name = "//div[@id='main']//div[@class='_21nHd']"
    body_text_area = "//footer[@class='_3E8Fg']//p"
    body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
    
    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        self.read_store = defaultdict()
        self.store_msg = pd.DataFrame([])
        print(self.driver.title)
    
    @property
    def get_driver(self):
        return self.driver
    
    def xpelem(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return None
    
    def xptext(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt).text
        except: return None
        
    def xpget(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt)
        except: return None
    
    def xpc(self, xpt):
        try: return wait.until(EC.element_to_be_clickable((By.XPATH, xpt)))
        except: return None
    
    def move_click(self, xpt):
        try: 
            self.ac.move_to_element(self.xpget(xpt)).click().perform()
            return 1
        except: return 0
        
    def hover_click(self, xp1, xp2):
        try: 
            self.ac.move_to_element(self.xpelem(xp1)).click(self.xpelem(xp2)).perform()
            return 1
        except: return 0
    
    def just_move(self, xp1):
        try: 
            self.ac.move_to_element(self.xpget(xp1)).perform()
            return 1
        except: return 0
        
    def paste_text(self, text):
        try: self.ac.send_keys(text).perform()
        except: pass
        
    def click_write_enter(self, xpt, text):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: return None
        xp.clear()
        if chr(10) not in text:
            self.ac.move_to_element(xp).click().send_keys(text).send_keys(Keys.RETURN).perform()
        else:
            xx = text.split(chr(10))
            for line in xx:
                self.ac.send_keys(line).perform()
                self.ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            self.ac.send_keys(Keys.RETURN).perform()
    
    def sidepane_scroll(self, initial = 1, scrolls=35):
        for i in range(1, scrolls):
            self.driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
            initial = initial + i + 1
        return initial

    def scroll_by_xpath(self, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
        try:
            xp = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", xp)
            return 1
        except:
            print('WhBase.scroll_by_xpath(xpt) failed to scroll')
            return 0
        
    def msg_sender_dttm(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        xpt = base + "[" + str(n) + "]"
        html = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).get_attribute('innerHTML')
        sp = BeautifulSoup(html, "lxml")
        x = sp.find_all('div')
        for i in x:
            try:
                st = i['data-pre-plain-text']
                break
            except:pass
        else: return None, None
        try:
            split_point = st.find(']')
            sender = st[split_point+1:].replace(':','').strip(' ')
            dttm = parse(st.upper()[1:split_point])
            return dttm.strftime("%Y-%m-%d %H:%M"), sender
        except: None, None
            
    def xptbulk(self, xpt):
        try:
            if type(xpt) is str: return self.xptext(xpt)
            elif type(xpt) is list: return [self.xptext(n) for n in xpt]
            elif type(xpt) is dict: return {k : [self.xptext(n) for n in v] if type(v) is list else self.xptext(v) for k, v in xpt.items()}
            else: return None
        except:
            return None
    
    def select_chat(self, chat_name):
        if chat_name is not None and chat_name != '':
            x = self.click_write_enter(xpt=self.pane_search, text=chat_name)
            y = self.xptext(self.body_chat_name)
            print('chat body name: ', y)
            if chat_name.lower() in y.lower():
                print('chat selection successful')
                return y
            else: 0
        else: return None
        
    def send_message(self, chat_names, text):
        cn = [chat_names] if type(chat_names) is str else chat_names
        for n in cn:
            x = self.select_chat(n)
            if x is not None or x !=0:
                self.click_write_enter(text)
                print('message send to ', str(n), ' sucessfully')
                tm.sleep(.5)
            else:
                print('message send failed, chat: ', str(n))
    
    def msgxp(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        else:
            self.move_click(bi + "//div[@role='button'][contains(.,'Read more')]")
            dttm, sender = self.msg_sender_dttm(n)
            dc = {'sender' : [sender],
                  'text' : [self.xptext(bi + "//div[@class='_21Ahp']/span[1]/span")],
                  'q_sender' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
                  'q_text' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
                  'datetime' : [dttm]}
            return dc
    
    def dict_to_df(self, dc, out=None):
        x = dt.datetime.now().strftime('%b%d%Y_%H%M')
        opt = x  + '.csv' if out is None else x + '_' + out + '.csv'
        pd.DataFrame.from_dict(dc).to_csv(opt)
        
    def forward_msg(self, bs, frwto='Onami'):
        self.move_click('/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span')
        tm.sleep(.5)
        if self.xpelem(bs) is not None:
            self.ac.move_to_element(self.xpelem(bs + "//div[@class='_2_-To']")).perform()
            tm.sleep(0.5)
            self.ac.click(self.xpelem(bs + "//span[@data-testid='down-context']")).perform()
            self.ac.move_to_element(self.xpelem("//div[@class='_2sDI2']//div[contains(text(),'Forward')]")).click().perform()
            self.ac.move_to_element(self.xpelem("//*[@data-testid='forward']")).click().perform()
            self.ac.send_keys(frwto).perform()
            self.ac.reset_actions()
            try:
                ActionChains(self.driver).move_to_element(self.xpelem("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")).click().perform()
            except:
                ActionChains(self.driver).click(self.xpelem("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")).perform()
                ActionChains(self.driver).move_to_element(self.xpelem('(//*[contains(concat( " ", @class, " " ), concat( " ", "_2N1Gm", " " ))])[1]')).click().perform()
            self.ac.move_to_element(self.xpelem("//*[@data-testid='send']")).click().perform()
            print('forwarded success')
        else:
            print('forward fail')

In [141]:
x = WhBase()
d = x.get_driver
ac = ActionChains(d)
wait = WebDriverWait(d, 5)

(3) WhatsApp


In [142]:
x.forward_msg("(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]", 'onami')



AttributeError: move_to requires a WebElement

In [131]:
ac.move_to_element(x.xpelem('(//*[contains(concat( " ", @class, " " ), concat( " ", "_2N1Gm", " " ))])[1]')).click().perform()

In [133]:
ac.move_to_element(x.xpelem("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")).click().perform()

In [ ]:
## bs = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"
x.xpelem(bs + "//span[@data-testid='down-context']")

In [ ]:
ac.move_to_element(x.xpelem("//ul[@class='_3bcLp']/div//*[contains(text(),'Forward')]")).click().perform()

In [ ]:
ac.move_to_element(x.xpelem("//*[@data-testid='forward']")).click().perform()

/html[@class='js serviceworker adownload cssanimations csstransitions webp exiforientation webp-alpha webp-animation webp-lossless wf-loading']/body[@class='web']/div[@id='app']/div[@class='_1Fm4m _1h2dM app-wrapper-web font-fix os-win']/span[2]/div[@class='lhggkp7q qq0sjtgm ebjesfe0 jxacihee tkdu00h0']/div[@class='snyj76hw qq0sjtgm tkdu00h0 cm280p3y ln8gz9je ppled2lx bgigc5s4 i44ccddp ag5g9lrv p1zdgkh6 _3cjY2 copyable-area']/div[@class='cm280p3y p357zi0d f8m0rgwh gndfcl4n ac2vgrno ln8gz9je s4r5ooj2 lffynu9d sdfuwbjb']/div[@class='cm280p3y p357zi0d ggj6brxn m0h2a7mj r6au09pb f8m0rgwh nhajnb67 i5tg98hk f9ovudaz przvwfww gx1rr48f gfz4du6o r7fjleex b19fvycv fe61fa5g qj4wrk6p cphhpnv8 tfm3omh7 paav9g0k g0rxnol2 kpwy2cir jnwc1y2a e2xccmyv svoq16ka agj3xlbq mtp0skmj h1tnk6gg ap935ont q1g5yed3 ciu2hny1 sz0fn9sj s7u82e91']/div[@class='g0rxnol2 ggj6brxn m0h2a7mj r6au09pb fd365im1 hmy10g0s']/div[@class='f8jlpxt4 iuhl9who']/div[@class='g6kkip0l p357zi0d f8m0rgwh ppled2lx tkdu00h0 gfz4du6o r7fjleex jv8uhy2r lhggkp7q qq0sjtgm ln8gz9je _3cjY2 copyable-area']/div[@class='g0rxnol2 thghmljt p357zi0d rjo8vgbg ggj6brxn f8m0rgwh gfz4du6o ag5g9lrv bs7a17vp']/div/div/div[@class='_2A1R8 _3YS_f']/div[@class='lhggkp7q ln8gz9je rx9719la'][2]/button[@class='i5tg98hk f9ovudaz przvwfww gx1rr48f shdiholb phqmzxqs gtscxtjd ajgl1lbb thr4l2wc cc8mgx9x eta5aym1 d9802myq e4xiuwjv g0rxnol2 ln8gz9je']/div[2]/div[@class='_199zF _3j691 _2N1Gm']

In [ ]:
x1 = lambda xp: wait.until(EC.presence_of_element_located((By.XPATH, xp)))
ac.move_to_element(x1("//ul[@class='_3bcLp']/div//*[contains(text(),'Forward')]")).click().perform()
ac.move_to_element(x1("//*[@data-testid='forward']")).click().perform()



In [ ]:
x.hover_click("(//div[@id='main']//div[@data-testid='msg-container'])[last()]", "(//div[@id='main']//div[@data-testid='msg-container'])[last()]//span[contains(@data-icon,'down-context')]")
x.move_click("//ul[@class='_3bcLp']/div//*[contains(text(),'Forward')]")
x.move_click("//*[@data-testid='forward']")

In [ ]:
ac.move_to_element(self.xpelem(xpt)).click().perform()

In [ ]:
dir(x)

In [ ]:
def cross(x1, x2=None, text=None):
    if x2 is None and text is None:
        ac.move_to_element(wait.until(EC.presence_of_element_located((By.XPATH, x1)))).perform()
    except:
        d.find_element(By.XPATH, xp).click()
            
        
def forward_message(x, to):
    msg =lambda x : "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[" + str(x) + "]"
    msg_arrow = lambda x :  msg(x) + "//div[@class='_3u9t-']/span"
    cross( msg('last()'))
    cross( msg_arrow('last()'))
    cross( "//ul[@class='_3bcLp']/div//*[contains(text(),'Forward')]" )
    cross("//button[@class='_5tHAy'][4]")
    cross({"(//div[contains(@role,'textbox')])[1]":to})
    cross("(//div[@class='_199zF _3j691 _2N1Gm'])[1]")
    cross("//span[contains(@data-icon,'send')]")

In [ ]:
x1 = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"
cross(x1)

In [ ]:
"//*[contains(concat( " ", @class, " " ), concat( " ", "_5tHAy", " " ))][4]"

In [ ]:
def bs4_attr_info(self, html, attr, attrvalue):
    sp = BeautifulSoup(html, "lxml")
    x = sp.find(attr=attrvalue)

In [ ]:
x = WhBase()
d = x.get_driver
dc = x.message_read('emergency', 500)

In [ ]:
print(dc)

In [ ]:


def msg_read(n, clsobj):
    dc, bi = msgxp(n)
    if clsobj.xpelem(bi) is None: return None
    else:
        #clsobj.just_move(bi)
        clsobj.move_click(dc['read_more_btn'])
        msgdttm, sender = clsobj.msg_sender_dttm(n)
        for k, v in dc.copy().items():
            if k == 'msg_sender': dc[k] = [sender]
            elif k == 'msg_datetime': dc[k] = [msgdttm]
            else: dc[k] = clsobj.xptbulk(v)
        return dc
    

In [ ]:
def msg_read_all(chat_name, clsobj=None):
    dic, ls = defaultdict(), []
    obj = x if clsobj is None else clsobj
    body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
    ret = obj.select_chat(chat_name)
    if ret is None or ret==0: 
        print('could not select chat, please check')
    else:
        duplicate, i, q = 2, 1, 0
        while duplicate==0 or duplicate<3:
            xdc = msg_read(i, clsobj)
            if xdc is not None and xdc['sender_text'] is not None:
                xy = '-'.join(map(str,xdc['sender_text'] + xdc['msg_datetime']))
                if xy not in ls:
                    q = q + 1
                    dic = update_key_value(dic, xdc)
                    ls.append(xy)
                    diplicate =0
                    print(q, xdc['msg_sender'],'-',xdc['msg_datetime'])
                    i = i + 1
                else:
                    i = 2
                    obj.just_move("(//div[@id='main']//div[@data-testid='msg-container'])[" + str(2) + "]")
                    tm.sleep(1)
            else:
                #obj.just_move("(//div[@id='main']//div[@data-testid='msg-container'])[" + str(1) + "]")
                obj.scroll_by_xpath()
                i = 2
                tm.sleep(1)
                duplicate = duplicate + 1
                print('dup', duplicate)
            
        print('while break')
    return dic

In [ ]:
dc = msg_read_all('emergency', x)


In [ ]:
df = pd.DataFrame.from_dict(dc)
print(df)

In [ ]:
df.to_csv('tx.csv')